# The environment
The environment of the game is **finite**, since we only have a finite number of possible values for each attribute, for example, the bird's velocity of falling down is an integer on an interval between -8 and 10.

There are eight parameters that represent a single state:  
* `player_y` (y coordinate of player's position)
* `player_vel` (player's velocity of falling down)
* `next_pipe_top_y` (y coordinate of the next pipe - upper part)
* `next_pipe_bottom_y` (y coordinate of the next pipe - lower part)
* `next_pipe_dist_to_player` (player's distance to the next pipe)
* `next_next_pipe_top_y` (y coordinate of the pipe after the next pipe - upper part)
* `next_next_pipe_bottom_y` (y coordinate of the pipe after the next pipe - lower part)
* `next_next_pipe_dist_to_player` (player's distance to the pipe after the next pipe)

Because the gap between two pipes (difference between `next_pipe_top_y` and `next_pipe_bottom_y`) is always the same, we can omit one of those attributes without losing accuracy. Furthermore, the difference between two successive sets of pipes stays the same, so we can ignore all the attributes concerning the pipe that is not the nearest one. We are then left with four attributes:  
* `player_y` (min: 0, max: 387)
* `player_vel` (min: -8, max: 10)
* `next_pipe_top_y` (min: 25, max: 192)
* `next_pipe_dist_to_player` (min: 0, max: 283)

This gives us $388 \cdot 19 \cdot 168 \cdot 284 = 351,732,864$ states, which is too much if we want to use them for learning, although a few of them are not reachable (the bird cannot be inside the pipe, for example). Therefore we discretisized the state space - we split each of the attributes except `player_vel` into $15$ intervals and compressed the state space to $19 \cdot 15^3 = 64,125$ states.

One thing to notice about the states and representing them, is that the x distance from pipe to next pipe is always the same. However at the beginning of a game, the bird is further away from the first pipe than what it'll be later. In other words variable `next_pipe_dist_to_player` starts with a value of $283$ at the beginning of the game, but after passing the first pipe, it's maximum value is $164$. A linear mapping between original states and discretized states is used. Formula for discretization is (int) (`original_value` $\cdot 14$ / $283$), which means that original states $0$-$20$ will discretized to $0$, original states $21$-$40$ will be discretized to $1$ etc. Original value of $164$ will be discretized to $8$, meaning that discretized values from $9$ to $14$ can be reached only in the first few frames of the game. This doesn't make much sense, because the agent shouldn't do anything special at the beginning. 

One way to handle this is to linearly discretize the original states between $164$ and $0$, and use one state for everything beyond that. Formula for this discretization is (int) (`original_value` $\cdot 14$ / $164$). If this mapping is used, there should be less states that are visited only at the beginning of the game. In this approach, position of the bird is known more accurately because the discretization is more dense. Both discretizations were tested. It seems that Monte Carlo learning gives better results when using the new, denser discretization. On the other hand, Q-Learning achieves better results with the original discretization. Because of that, we used different environments for Monte Carlo learning and for Q-Learning.

The only **stochastic** thing in the environment is the position of the gap between two pipes. Everything else is **deterministic**: the speed of the bird in x direction and the distance between two successive sets of pipes are constant, after flapping the bird moves up (always for the same height) and after doing nothing the bird moves down with a fixed acceleration, so we can always calculate the next state (except `next_pipe_top_y` if the bird just successfully flied throug the gap). Besides that, collisions (bird - pipe) are always calculated in the same way. In other words, whenever the current state is `s` and `a` is a chosen action, the successor state that we end up in is the same.

The environment is **episodic**, although an episode has a possibility to be infinite. A single episode starts when the game starts and ends when the bird flies into a pipe.

The process is a **Markov decision process** because a selection of the action can be made independent from the states that were visited before entering the current state if we know the values for each of the four attributes listed above. With discretization the number of states becomes controllable, but the Markov property is lost. Now each state represents a set of states and it can be important, which _original_ state we are in to select the best action. However, this simplification still allows our agent to learn a good policy.

## Algorithms (not) to be used
There are multiple possible algorithms that can be used to learn a policy for given environment, but some of them are more appropriate for this task than others.
* **Dynamic Programming**  
  The environment is finite and (almost) deterministic so it would be possible to make a model to represent it. However, the state space is huge and stochastic heights of the pipes would make it fairly complicated. Furthermore, violating Markov property with discretization has a negative effect on this approach.
* **Monte Carlo**  
  Using Monte Carlo algorithm to learn a policy requires episodic environment and our environment has this property. Besides that, discretization makes less harm here. When implementing this algorithm some testing is needed to determine what parameter values (e. g. learning rate, discount rate and epsilon) should be chosen to achieve optimal results.
* **Q-learning and Sarsa**  
  Temporal-difference algorithms are even less demanding as Monte Carlo. They do not require episodic tasks, need less memory and are much faster than Dynamic Programming or Monte Carlo algorithm. As with Monte Carlo, we have to test different parameters to find the best configuration.

# On-policy Monte Carlo control

A basic implementation of On-policy Monte Carlo control was tested with following values:
- Discounting = 1.0 (no discounting)
- Learning rate = 0.1
- Epsilon = 0.1

Without discounting the agent doesn't seem to learn anything, so discounting of 0.9 was added without changing other parameters. Two agents were tested. Agent A doesn't use discounting and agent B uses discounting of 0.9.

Below is the learning curve showing progress of training over 10000 episodes. It clearly shows that such values of parameters are quite bad. Especially the lack of discounting seems to be the biggest problem. The agents learn very slowly, high variance is observed. There is a lot to improve.

![](plots/average_A_B_10000ep.jpg)

# Q-Learning

A basic implementation of Q-Learning was tested with following values:
- Discounting = 1.0 (no discounting)
- Learning rate = 0.1
- Epsilon = 0.1

Plot below shows the learning progress over 20 000 training episodes. Agent's playing was evaluated after every 1 000 episodes. For evaluation, the agent played 100 times using the real policy and average of those scores is used in the plot.


![](plots/average_QLearning_20000ep.jpg)

Agent starts with an average score of zero, which means that most of the times, it doesn't get through a single pipe. As the agent is trained, it gets better in the game and achieves a better score. The first 4000 iterations are quite bad but after that the agent starts to get better. After 8000 iterations the average score is only 5, but after 12 000 iterations the average score is 20, which is already quite good.

After 12 000 episodes the evaluation score gets both better and worse, which is a weird result. Plot is averaging 100 evaluations so that should be enough to decrease the effect of randomness. One explanation for the results could be, that the learning rate of 0.1 is too big, and should be decreased to get less variance in the evaluation score. 

# Comparison of Monte Carlo and Q-Learning

The results of the Monte Carlo agents and Q-Learning agent can be plotted in the same plot to compare them. We used a slightly different environment for these different agents because that gave better results on both agents.

- A - Monte Carlo, no discounting
- B - Monte Carlo, discounting = 0.9
- C - Q-Learning

![](plots/average_A_B_C_10000ep.jpg)

Monte Carlo without discounting doesn't seem to work. Discounted Monte Carlo and Q-Learning can be compared. It seems that Monte Carlo gets better scores before 5000 episodes, but after 7000 episodes Q-Learning achieves a better score. At the beginning Q-Learning is slower to learn because the rewards propagate back slower than in Monte Carlo. After 7000 iterations Q-Learning gets better results because it's learning off-policy, which means that it can learn a greedy policy. Monte Carlo is on-policy and it's learning an epsilon-greedy policy which isn't as good.

# Linear function approximation
Linear function approximation using eight weights is not appropriate for learning Q-values or policy since neither of these functions is linear. No matter what linear function is chosen, there will be some good predictions but the majority of predictions will be very wrong.

When we tried to implement that (code in *function_approximation.py*), we noticed that the numbers become positive or negative infinity after only a few episodes, which confirms our expectations that this method does not work.

But there is a workaround that makes learning possible using this aproximation. The number of weights should be increased - there should be a weight for each of the (discretisized) states in the environment. The vector $\phi(s)$ should have the same number of values and each of them is either $0$ (if the agent is in the state represented by this index) or $1$ (if the agent is not in this state). Then it is possible to learn the Q-values and policy with linear function approximation of those features.

# Make the best agent

The agent chosen for the expretimets is the one trained with on-policy Monte Carlo control approach, as during the course of experimentation it was the agent which scores proved to be the highest after 10000 episodes with decent parameters.

First parameter to be tuned was **discounting**. Experiments showed that too high values (above 0.98) give effects close to no discounting (very poor scores) and too low values cause the information about the success or failure (reward) not to be able to propagate properly through state space resulting with poor scores. Optimal value of 0.85 was found. 

Next paramet was **epsilon**. With low values the agent learns very slowly because he is constantly repeating the same actions and does not realize that another action would bring him a better return. On the other hand, too high values cause the agent making random actions more often and therefore he learns accurate values for more states, but that also lenghtens the learning. Experiments showed that constant value is not necessarily the best way to go, so the mechanism to reduce this value during the course of training was introduced. Starting values of 0.05 for epsilon and 0.1 for **learning rate** proved to be quite good. 

Lastly the **reward structure** was expanded to incentivise certaing behaviour - mainly for the agent to stay on the level of the gap. The further away from the gap, the bigger the penalty, especially in the case of moving (velocity) in the wrong direction (i.e. gap is below the player and the velocity indicate moving upwards). Optimal position of being at the level of the center of the gap with velocity values indicating little vertical movement give the highest reward. Its value also had to be tuned, as too low would not have affect on the outcome, and too high would overshadow effects of passing through pipe or hitting the wall. Value of 0.07 proved to be quite good. 

Below is a couple of learning curves for different values of parameters tried during tunig process.

![](plots/average_164_5000ep.jpg)

Agent with too low discounting (0.6) and too high value for extended reward system parameter. 

![](plots/average_164_5000ep3.jpg)

Agent with too high epsilon of 0.2 and learning rate of 0.4

![](plots/average_164_5000ep2.jpg)

Agent with finally tuned epsilon, learning rate, discount factor and extended reward system parameter.

# Let's win
![](plots/average_164_10000ep.jpg)

The plot shows results of our final agent after 10000 iterations with added system to decrease epsilon over time and additionaly lessen the effect of extended reward system right after passing through pipe to avoid situations of barely hitting the pipe while trying to get to correct level.

Parameters chosen:  
* Learning rate = 0.1
* Discounting = 0.85
* Starting epsilon = 0.05
* Starting reward factor = 0.07

State representation is the same as used in original algorithm.

This agent should be used for both tasks (best policy and quickest policy).

The code is in file `montecarlo_agent_best.py`, and the class name is `MonteCarloAgentBest`.